In [ ]:
from python_simulation_manager.experiment_base.experiment_handler import ExperimentHandler
from python_simulation_manager.experiment_base.experiment_output import ExperimentOutput
from typing import override
import numpy as np

In [56]:
class MyTestOutput(ExperimentOutput):
    def __init__(self, file_name):
        super().__init__(file_name)
        self.hello_world     = None
        self.sum            = None
        self.multiplication = None

    @override
    def parse_output(self, line_number, lines):
        if line_number == 0:
            self.hello_world = lines.removesuffix("\n")
        else:
            slines = lines.split(", ")
            self.sum            = float(slines[0])
            self.multiplication = float(slines[1])

class MyTestExperiment(ExperimentHandler):

    @override
    def set_result_type(self, output_file) -> MyTestOutput:
        return MyTestOutput(output_file)
                
class TestExperimentBuilder:
    def __init__(self, name: str, folder: str, rust_dir: str):
        self.name: str     = name
        self.folder: str   = folder
        self.rust_dir: str = rust_dir

    def build_experiment(self: str, my_array: np.array, my_bool: bool, steps: dict):
     
        new_exp = MyTestExperiment(self.name, self.folder, self.rust_dir)
        new_exp.add_static_parameter("my_array", my_array)
        new_exp.add_static_parameter("my_bool", my_bool)
        new_exp.add_scaling_parameter("steps", steps)
        new_exp.set_scaling_name(["Lx", "Ly"])
        return new_exp

In [57]:
test_builder      = TestExperimentBuilder(name="test_experiment", folder="test_results", rust_dir="rust_example")
an_array          = np.arange(0, 10, 1)
a_boolean         = True
monte_carlo_steps = {
    2: 100,
    4: 1_000,
    8: 1_000_000
}

test_exp = test_builder.build_experiment(an_array, a_boolean, monte_carlo_steps) 
test_exp.write_parameter_files()


test_exp.set_static_parameter("my_bool", False)

# if not test_exp.are_parameter_files_available():


>> Directory "rust_example/test_results/test_experiment" Found.
>> Succesfully added my_array
>> Succesfully added my_bool
>> Succesfully added steps
>> Setting file paths:
-- setting paramfile "test_results/test_experiment/parameter_2x2.txt"
-- setting paramfile "test_results/test_experiment/parameter_4x4.txt"
-- setting paramfile "test_results/test_experiment/parameter_8x8.txt"
-- setting outfile "test_results/test_experiment/out_2x2.txt"
-- setting outfile "test_results/test_experiment/out_4x4.txt"
-- setting outfile "test_results/test_experiment/out_8x8.txt"
>> writing parameter files:
-- test_results/test_experiment/parameter_2x2.txt: done
-- test_results/test_experiment/parameter_4x4.txt: done
-- test_results/test_experiment/parameter_8x8.txt: done
>> "my_bool" set


In [58]:
for L in test_exp.get_lengths():
    # if not test_exp.has_output(L):
    test_exp.perform_rust_computation(L)
    print("")

Command: "cargo run --release -- test_results/test_experiment/parameter_2x2.txt"
 * From Rust:     Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.03s
 * From Rust:      Running `target\release\rust_example.exe test_results/test_experiment/parameter_2x2.txt`
 * From Rust: Reading parameters:
 * From Rust: Lx: 2
 * From Rust: Ly: 2
 * From Rust: my_array: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
 * From Rust: my_bool: True
 * From Rust: steps: 100
 * From Rust: outputfile: test_results/test_experiment/out_2x2.txt
 * From Rust: Writing in test_results/test_experiment/out_2x2.txt

Command: "cargo run --release -- test_results/test_experiment/parameter_4x4.txt"
 * From Rust:     Finished ]8;;https://doc.rust-lang.org/cargo/reference/profiles.html#default-profiles\`release` profile [optimized]]8;;\ target(s) in 0.03s
 * From Rust:      Running `target\release\rust_example.exe test_results/test_experiment/par

In [59]:
test_exp.are_results_available()

>> Looking for output files:
-- Found output: "rust_example/test_results/test_experiment/out_2x2.txt"
-- Found output: "rust_example/test_results/test_experiment/out_4x4.txt"
-- Found output: "rust_example/test_results/test_experiment/out_8x8.txt"


True

In [66]:
results = test_exp.get_results()
for (Lx, result) in results.items():
    print(f"For Lx = {Lx} we have:")
    print(result.hello_world)
    print(f"Sum            = {result.sum}")
    print(f"multiplication = {result.multiplication}")



Found output: "rust_example/test_results/test_experiment/out_2x2.txt
Found output: "rust_example/test_results/test_experiment/out_4x4.txt
Found output: "rust_example/test_results/test_experiment/out_8x8.txt
For Lx = 2 we have:
Hello world! [Lx=2]
Sum            = 4.0
multiplication = 4.0
For Lx = 4 we have:
Hello world! [Lx=4]
Sum            = 8.0
multiplication = 16.0
For Lx = 8 we have:
Hello world! [Lx=8]
Sum            = 16.0
multiplication = 64.0


In [51]:
from python_simulation_manager.monte_carlo_ising_for_rust import RustExperimentBuilder, RustIsingExperiment


In [52]:
thermalization_steps = {2: 1_000_000, 4: 500_000,  8: 100_000}
measure_steps        = {2: 1_000_000, 4: 500_000,  8: 100_000}
temperatures         = np.arange(0, 10, 1)
builder              = RustExperimentBuilder(name="ising_overview", folder="results", rust_dir="ising_rust")

experiment = builder.new_from_parameters(thermalization_steps, measure_steps, temperatures)
experiment.write_parameter_files()

>> Directory "ising_rust/results/ising_overview" Found.
>> Succesfully added temperatures
>> Succesfully added measure_struct_fact
>> Succesfully added therm_steps
>> Setting file paths:
-- setting paramfile "results/ising_overview/parameter_2x2.txt"
-- setting paramfile "results/ising_overview/parameter_4x4.txt"
-- setting paramfile "results/ising_overview/parameter_8x8.txt"
-- setting outfile "results/ising_overview/out_2x2.txt"
-- setting outfile "results/ising_overview/out_4x4.txt"
-- setting outfile "results/ising_overview/out_8x8.txt"
>> Succesfully added measure_steps
>> writing parameter files:
-- results/ising_overview/parameter_2x2.txt: done
-- results/ising_overview/parameter_4x4.txt: done
-- results/ising_overview/parameter_8x8.txt: done


In [53]:
for L in experiment.get_lengths():
    experiment.perform_rust_computation(L)

Command: "cargo run --release -- results/ising_overview/parameter_2x2.txt"
 * From Rust: error: could not find `Cargo.toml` in `c:\Users\solof\Desktop\programming\python\GITHUB\python_simulation_manager\ising_rust` or any parent directory
Command: "cargo run --release -- results/ising_overview/parameter_4x4.txt"
 * From Rust: error: could not find `Cargo.toml` in `c:\Users\solof\Desktop\programming\python\GITHUB\python_simulation_manager\ising_rust` or any parent directory
Command: "cargo run --release -- results/ising_overview/parameter_8x8.txt"
 * From Rust: error: could not find `Cargo.toml` in `c:\Users\solof\Desktop\programming\python\GITHUB\python_simulation_manager\ising_rust` or any parent directory


In [ ]:
experiment.get_results()